<b> Source COVID World updated everyday at 2:00 AM : </b> <br> 
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series <br>

In [1]:
%run __init__.ipynb

Successfully connected to MongoDB


In [2]:
def url_to_csv_mongo(url,df_name):
    df = check_url(url)
    if not df.empty:
        # Save in CSV
        df_save(df, df_name,'csv')
        
        # Save in MongoDB
        if USE_MONGO:
            naas_drivers.mongo.send(df, df_name, DB_SRC,True)
    else:
        print(f'DataFrame {df_name} is empty !')
    return df

In [3]:
# url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
# df = pd.read_csv(url, sep=',')
# df

In [4]:
%%time

#--- STEP1 ---: Get data source & cleaning

# RACINE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'

#-- GITHUB : TIMESERIES (CSSEGISandData/COVID-19/csse_covid_19_time_series)
#-> WORLD_CONFIRMED
urlConfirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
cConfirmed = url_to_csv_mongo(urlConfirmed,'WORLD_CONFIRMED')

#-> WORLD_DEATHS
urlDeath = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
cDeath = url_to_csv_mongo(urlDeath,'WORLD_DEATHS')

#-> WORLD_RECOVERED
urlRecovery = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
cRecovery = url_to_csv_mongo(urlRecovery,'WORLD_RECOVERED')

#-> US_CONFIRMED
urlConfirmedUS = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
cConfirmedUS = url_to_csv_mongo(urlConfirmedUS,'US_CONFIRMED')

#-> US_DEATHS
urlDeathUS = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
cDeathUS = url_to_csv_mongo(urlDeathUS,'US_DEATHS')

#-- GITHUB : DAILY REPORT (CSSEGISandData/COVID-19/csse_covid_19_time_series)
#-> DAILY
date_init = YESTERDAY.strftime('%m-%d-%Y')
date_end = YESTERDAY.strftime('%m-%d-%Y')
dates = pd.date_range(start=date_init,end=date_end, freq='D').strftime('%m-%d-%Y').values.tolist()
filter_dates = pd.date_range(start=date_init,end=date_end, freq='D').strftime('%m/%d/%y').values.tolist()

cDaily = pd.DataFrame()
for d in dates:
    'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    urlDaily = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + d + '.csv'
    cols_to_rename = {"Country/Region": "Country_Region","Province/State": "Province_State", "Long_":"Long"}
    tmp_df = check_url(urlDaily).rename(index=str, columns=cols_to_rename)
    if len(tmp_df) != 0:
        tmp_df['Confirmed'] = tmp_df['Confirmed'].fillna(0).astype(int)
        tmp_df['Deaths'] = tmp_df['Deaths'].fillna(0).astype(int)
        tmp_df['Recovered'] = tmp_df['Recovered'].fillna(0).astype(int)
        tmp_df['Date'] = pd.to_datetime(d, format='%m-%d-%Y').strftime('%m/%d/%y')
        cDaily = cDaily.append(tmp_df)

WORLD_CONFIRMED successfully saved in WORLD_CONFIRMED
Dataframe WORLD_CONFIRMED successfully save in database covid-19 in MongoDB. Time: --- 0.15373945236206055 secnds ---
WORLD_DEATHS successfully saved in WORLD_DEATHS
Dataframe WORLD_DEATHS successfully save in database covid-19 in MongoDB. Time: --- 0.1858377456665039 secnds ---
WORLD_RECOVERED successfully saved in WORLD_RECOVERED
Dataframe WORLD_RECOVERED successfully save in database covid-19 in MongoDB. Time: --- 0.14579057693481445 secnds ---
US_CONFIRMED successfully saved in US_CONFIRMED
Dataframe US_CONFIRMED successfully save in database covid-19 in MongoDB. Time: --- 6.322901248931885 secnds ---
US_DEATHS successfully saved in US_DEATHS
Dataframe US_DEATHS successfully save in database covid-19 in MongoDB. Time: --- 6.366649150848389 secnds ---
CPU times: user 5.47 s, sys: 276 ms, total: 5.74 s
Wall time: 14.5 s


In [5]:
%%time

#-- Melt data source from timeseries to have date in rows
def melt_cleaning(df,variable):
    try:
        if 'Province/State' and 'Country/Region' in df.columns:
            #Add Country/Region to Province/State if empty
            df = df.fillna(0)
            df.loc[df['Province/State'] == 0,'Province/State'] = df['Country/Region']
            #Melt and group data
            cols_to_keep = ['Country/Region','Province/State']
            cols_to_group = ['Country/Region','Province/State','Date']
            df = pd.melt(df, id_vars=cols_to_keep,value_name=variable,var_name='Date')
            df = df.groupby(cols_to_group, as_index=False).agg({variable:'sum'})
            #Data format
            df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')

            if variable == 'Confirmed':
                df['Deaths'] = 0
                df['Recovered'] = 0
            elif variable == 'Deaths':
                df['Confirmed'] = 0
                df['Recovered'] = 0
            else:
                df['Confirmed'] = 0
                df['Deaths'] = 0
        else:
            df=pd.DataFrame()
    except Exception as e:
        df=pd.DataFrame()
        print(e.__doc__)
        print(str(e))  
    return df

def cols_cleaning(df,cols_to_drop):
    for c in cols_to_drop:
        if c in df.columns:
            df = df.drop(c,axis=1)
            
    cols_to_rename = {"Province_State": "Province/State", "Country_Region": "Country/Region"}
    df = df.rename(index=str, columns=cols_to_rename)
    return df       

# Timeseries : World Confirmed
cols_to_drop = ['Lat','Long']
cConfirmed=cConfirmed[cConfirmed['Country/Region'] != 'US']
df_cW = melt_cleaning(cols_cleaning(cConfirmed, cols_to_drop),'Confirmed')
    
# Timeseries : World Deaths
cols_to_drop = ['Lat','Long']
cDeath=cDeath[cDeath['Country/Region'] != 'US']
df_dW = melt_cleaning(cols_cleaning(cDeath, cols_to_drop),'Deaths')
    
# Timeseries : World Recovered
cols_to_drop = ['Lat','Long']
df_rW = melt_cleaning(cols_cleaning(cRecovery, cols_to_drop),'Recovered')

# Timeseries : US Confirmed
cols_to_drop = ['UID','iso2', 'iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key']
df_cUS = melt_cleaning(cols_cleaning(cConfirmedUS, cols_to_drop),'Confirmed')

# Timeseries : US Deaths
cols_to_drop = ['UID','iso2', 'iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key','Population']
df_dUS = melt_cleaning(cols_cleaning(cDeathUS, cols_to_drop),'Deaths')

# Manual input
df_manual = pd.read_excel(REF_WSR_PATH, sheet_name = 'DATA_INPUT').fillna(0)
df_manual['Date'] = pd.to_datetime(df_manual['Date'], format='%Y%m%d').dt.strftime('%m/%d/%y')

CPU times: user 2.08 s, sys: 112 ms, total: 2.19 s
Wall time: 2.38 s


In [6]:
%%time

#--- STEP2 ---: Create DB with data source (output = DB_CONCAT)
def step2(df1,df2,df3,df4,df5,df6):
    #-- Merge timeseries
    cols_to_group = ['Country/Region','Province/State','Date']
    df = pd.concat([df1,df2,df3,df4,df5,df6], axis=0).groupby(cols_to_group, as_index=False).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'})
    return df.reset_index(drop=True)

db_concat = step2(df_cW,df_dW,df_rW,df_cUS,df_dUS,df_manual)
df_save(db_concat, 'WORLD_DB_CONCAT','csv')
# db_concat

WORLD_DB_CONCAT successfully saved in WORLD_DB_CONCAT
CPU times: user 486 ms, sys: 23.4 ms, total: 509 ms
Wall time: 625 ms


In [7]:
%%time

#--- Check if date is updated
df_check = db_concat[db_concat['Date'] == YESTERDAY.strftime('%m/%d/%y')]
continue_script = True
if len(df_check) == 0:
    print(f'Data not updated !')
    continue_script = False
else:
    print(f"Data updated, {YESTERDAY} exits in WORLD_DB_CONCAT")

Data updated, 2020-11-03 exits in WORLD_DB_CONCAT
CPU times: user 10 ms, sys: 137 µs, total: 10.2 ms
Wall time: 10.1 ms


In [8]:
%%time

#--- STEP3 ---: Consolidate and enrich data (output = DB_CONSO)
#-> Referentials
ref_continent = pd.read_excel(REF_WSR_PATH, sheet_name = 'REF_CONTINENT')
def step3(df,ref):
    df.columns = df.columns.str.upper()
    cols_to_rename = {"COUNTRY/REGION": "ENTITY_GROUPS", "PROVINCE/STATE": "ENTITY"}
    df = df.rename(index=str, columns=cols_to_rename)
    
    cols_to_rename = {"ENTITY_GROUPS": "ENTITY", "CONTINENT": "ENTITY_GROUPS"}
    df_con = pd.merge(df.drop(['ENTITY'],axis=1),ref.drop(['WORLDMAP'],axis=1), left_on=['ENTITY_GROUPS'], right_on=['COUNTRY_REGION'], how='left').rename(index=str, columns=cols_to_rename).drop(['COUNTRY_REGION'],axis=1).fillna(0)
    df_con.loc[df_con['ENTITY_GROUPS'] == 0, 'ENTITY_GROUPS'] = 'To be affected'
    
    cols_to_rename = {"ENTITY_GROUPS": "ENTITY"}
    df_ww = df_con.copy().drop(['ENTITY'],axis=1).rename(index=str, columns=cols_to_rename)
    df_ww['ENTITY_GROUPS'] = 'WORLDWIDE'
    
    cols_to_rename = {"ENTITY_GROUPS": "ENTITY"}
    df_tt = df_ww.copy().drop(['ENTITY'],axis=1).rename(index=str, columns=cols_to_rename)
    df_tt['ENTITY_GROUPS'] = 'WORLDWIDE'

    df = pd.concat([df,df_con,df_ww,df_tt], axis=0)
    df = df.groupby(['ENTITY_GROUPS','ENTITY','DATE'], as_index=False).agg({'CONFIRMED': 'sum','DEATHS': 'sum', 'RECOVERED': 'sum'})
    return df.reset_index(drop=True)

if continue_script:
    db_conso = step3(db_concat,ref_continent)
    df_save(db_conso, 'WORLD_DB_CONSO','csv')
# db_conso

WORLD_DB_CONSO successfully saved in WORLD_DB_CONSO
CPU times: user 1.13 s, sys: 59.3 ms, total: 1.19 s
Wall time: 1.48 s


In [9]:
%%time

#--- STEP4 ---: Calculate KPIs (output = DB_ALL)
#-> Referentials
ref_tech = pd.read_excel(REF_WSR_PATH, sheet_name = 'PARAM_ALL')

def step4(df,ref):
    #-- Calc Active + Ratio
    df['ACTIVE_CASES'] = df['CONFIRMED'] - df['DEATHS'] - df['RECOVERED']
    df.loc[:,'DEATHS_RATIO'] = 0
    df.loc[df['CONFIRMED'] != 0, 'DEATHS_RATIO'] = df['DEATHS'] / df['CONFIRMED'] * 100
    df.loc[:,'RECOVERED_RATIO'] = 0
    df.loc[df['CONFIRMED'] != 0, 'RECOVERED_RATIO'] = df['RECOVERED'] / df['CONFIRMED'] * 100
    
    #-- Melt KPI in rows
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','DATE']
    df= df.melt(id_vars=cols_to_keep,value_name='VALUE',var_name='KPI')
    indexes = df.loc[df['DATE'] == 0].index
    df = df.drop(indexes, axis=0)
    
    #-- Variation vs last day
    #Add fields
    df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%y')
    df['LAST_DAY'] =  pd.to_datetime(df['DATE'] + timedelta(days=-1))
    
    #Create new df
    cols_to_rename = {"DATE": "LAST_DAY",'VALUE':'VALUE_D-1'}
    df_last = df.drop(['LAST_DAY'], axis=1).rename(index=str, columns=cols_to_rename)
    
    #Merge day-1
    cols_to_merge = ['ENTITY_GROUPS','ENTITY','LAST_DAY','KPI']
    df = df.merge(df_last, on=cols_to_merge, how='left').fillna(0).drop(['LAST_DAY'], axis=1)
    
    #Calc variation in value and %
    df['VARV'] = df['VALUE'].astype(float) - df['VALUE_D-1'].astype(float) 
    df.loc[:,'VARP'] = np.NaN
    df.loc[(df['KPI'].isin(['CONFIRMED','DEATHS','RECOVERED','ACTIVE_CASES'])) & (df['VALUE_D-1'] != 0), 'VARP'] = df['VARV'] * 100 / abs(df['VALUE_D-1'])
    
    #-- Add fields
    df['SCENARIO'] = pd.to_datetime(df['DATE'], format='%m/%d/%y').dt.strftime('%d/%m/%Y')
    df['DATE_ORDER'] = pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y').dt.strftime('%Y%m%d')
    
    #-- Units / Precisions / Sentiments 
    df['KPI'] = df['KPI'].str.replace('_',' ').str.lower().str.capitalize()
    df= pd.merge(df,ref, on=['KPI'], how='left')
    df.loc[df['KPI'] == 'Deaths ratio', 'KPI'] = "Fatality Rate"
    df.loc[df['KPI'] == 'Recovered ratio', 'KPI'] = "Recovery Rate"
    return df.reset_index(drop=True)

if continue_script:
    db_all = step4(db_conso,ref_tech)
    db_all = optimize(db_all, ['DATE'])
    df_save(db_all, 'WORLD_DB_ALL','csv')
# db_all

WORLD_DB_ALL successfully saved in WORLD_DB_ALL
CPU times: user 22.6 s, sys: 614 ms, total: 23.3 s
Wall time: 25.9 s


In [13]:
%%time

#--- STEP5 ---: Transform DB_ALL to trend data (output = DB_TREND)
#-> Referentials
ref_tech = pd.read_excel(REF_WSR_PATH, sheet_name = 'PARAM_TREND')

def step5(df,ref):
    #-- Melt CALC in rows
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','SCENARIO','DATE','DATE_ORDER','KPI']
    df = df.drop(['VALUE_D-1','UNIT_VALUE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP'],axis=1)
    df = pd.melt(df, id_vars=cols_to_keep, value_name='VALUE',var_name='METRIC')
    df['DATE_SCENARIO'] = 'Since beginning'
        
    df_last14 = get_lastdays(df, 14,"Last 14 days")
    df_last30 = get_lastdays(df, 30,"Last 30 days")
    df = pd.concat([df,df_last14,df_last30],axis=0)
    
    #-- Get units and precisions
    df= pd.merge(df,ref, on=['KPI','METRIC'], how='left')
    df.loc[df['KPI'] == 'Deaths ratio', 'KPI'] = "Fatality Rate"
    df.loc[df['KPI'] == 'Recovered ratio', 'KPI'] = "Recovery Rate"
    df = df.reset_index(drop=True)
    return df

if continue_script:
    db_trend = step5(db_all,ref_tech)
    db_trend = optimize(db_trend)
    df_save(db_trend, 'WORLD_DB_TREND','csv')

WORLD_DB_TREND successfully saved in WORLD_DB_TREND
CPU times: user 51.6 s, sys: 1.04 s, total: 52.7 s
Wall time: 1min 44s


In [11]:
#--- STEP6 ---: Check and validate data
#-> CHECK TIMESERIES
#change yesterday format to match with header

def check_and_validate():
    YESTERDAY_C = YESTERDAY.strftime('%m/%d/%y')
    if YESTERDAY_C[:1]== '0':
        YESTERDAY_C = YESTERDAY_C[1:].replace('/0','/')

    #check if yesterday exist in df
    def check_timeseries(date,df,df_name):
        if date in df.columns:
            check = df.agg({date:'sum'}).fillna(0)[0]
        else:
            check = 0
            print(f'Column {date} does not exist in {df_name}.\n')
        return check

    check_ct = check_timeseries(YESTERDAY_C,cConfirmed, 'CONFIRMED') + check_timeseries(YESTERDAY_C,cConfirmedUS, 'CONFIRMED')
    check_dt = check_timeseries(YESTERDAY_C,cDeath, 'DEATHS') + check_timeseries(YESTERDAY_C,cDeathUS, 'DEATHS')
    check_rt = check_timeseries(YESTERDAY_C,cRecovery, 'RECOVERED')
    check_at = check_ct - check_dt - check_rt
    if check_ct != 0:
        check_frt = check_dt / check_ct * 100
        check_rrt = check_rt / check_ct * 100
    else:
        check_frt = 0
        check_rrt = 0

    #-> CHECK DAILY
    #check if yesterday exist in df
    def check_timeseries(date_check,df,variable):
        if len(df) != 0 and "Date" in df.columns and variable in df.columns and type(date_check) is date:
            check = df[df['Date'] == date_check.strftime('%m/%d/%y')].agg({variable:'sum'})[0]
        else:
            check = 0
            print(f'No data in Daily\n')
        return check

    check_cd = check_timeseries(YESTERDAY,cDaily,'Confirmed')
    check_dd = check_timeseries(YESTERDAY,cDaily,'Deaths')
    check_rd = check_timeseries(YESTERDAY,cDaily,'Recovered')
    check_ad = check_cd - check_dd - check_rd
    if check_cd != 0:
        check_frd = check_dd / check_cd * 100
        check_rrd = check_rd / check_cd * 100
    else:
        check_frd = 0
        check_rrd = 0

    #-- CHECK : TIMESERIES vs DAILY
    check_c = check_ct - check_cd
    check_d = check_dt - check_dd
    check_r = check_rt - check_rd
    check_a = check_at - check_ad
    check_fr = check_frt - check_frd
    check_rr = check_rrt - check_rrd

    print(f'Data check {YESTERDAY}: TimeSeries - Daily')
    print(f'Confirmed: {check_ct} - {check_cd} = {check_c}')
    print(f'Deaths: {check_dt} - {check_dd} = {check_d}')
    print(f'Recovered: {check_rt} - {check_rd} = {check_r}')
    print(f'Active cases: {check_at} - {check_ad} = {check_a}')
    print(f'Fatality rate: {check_frt} - {check_frd} = {check_fr}')
    print(f'Recovery rate: {check_rrt} - {check_rrd} = {check_rr}')
    
if continue_script:
    check_and_validate()

Column 11/03/20 does not exist in CONFIRMED.

Column 11/03/20 does not exist in CONFIRMED.

Column 11/03/20 does not exist in DEATHS.

Column 11/03/20 does not exist in DEATHS.

Column 11/03/20 does not exist in RECOVERED.

Data check 2020-11-03: TimeSeries - Daily
Confirmed: 0 - 47405395 = -47405395
Deaths: 0 - 1213735 = -1213735
Recovered: 0 - 31609242 = -31609242
Active cases: 0 - 14582418 = -14582418
Fatality rate: 0 - 2.560330949673555 = -2.560330949673555
Recovery rate: 0 - 66.67857529717874 = -66.67857529717874
